참고: https://revf.tistory.com/280

# OpenAI (text-davinci-003) 활용 RAG 기반 다국어 통역 성능 테스트

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

In [4]:
import json

In [10]:
with open("./secrets.json") as f:
    SECRETS = json.load(f)
    
openai_api_key = SECRETS['OPENAI_KEY']

In [11]:
chat = ChatOpenAI(temperature=.7, openai_api_key=openai_api_key)

# Schema

## 채팅메시지

In [12]:
chat(
    [
        SystemMessage(content="당신은 금융 통역 AI 봇입니다. 주어진 문장을 영어로 자연스럽게 번역해주세요."),
        HumanMessage(content="한도제한계좌를 해지하러 왔습니다.")
    ]
)

AIMessage(content="I'm here to cancel my account with a limit.", additional_kwargs={}, example=False)

## 문서

In [13]:
from langchain.schema import Document

In [14]:
Document(page_content="한도제한계좌란 입출금이 자유로운 요구불계좌 신규 시 금융거래 목적에 대한 객관적인 증빙서류가 제출되지 않아 일반계좌 개설이 불가능한 경우에 개설되는 계좌이다.",
        metadata={
            'my_document_id': 1,
            'my_document_source': 'google search',
            'my_document_create_time': 1680013019
        },
        lookup_index=0)

Document(page_content='한도제한계좌란 입출금이 자유로운 요구불계좌 신규 시 금융거래 목적에 대한 객관적인 증빙서류가 제출되지 않아 일반계좌 개설이 불가능한 경우에 개설되는 계좌이다.', metadata={'my_document_id': 1, 'my_document_source': 'google search', 'my_document_create_time': 1680013019})

In [15]:
# from langchain.document_loaders import PyPDFLoader

# Models

## template 이용

In [26]:
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate

In [28]:
template="당신은 {input_language}를 {output_language}로 번역하는 유용한 금융 전문 통역봇입니다."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

In [31]:
# 형식이 지정된 메시지에서 채팅 완료 가져오기
chat(
    chat_prompt.format_prompt(
        input_language="Korean",
        output_language="English",
        text="한도제한계좌의 한도를 해지하고 싶습니다."
    ).to_messages()
)

AIMessage(content='I would like to cancel the limit of my account.', additional_kwargs={}, example=False)

## 텍스트 임베딩 모델

In [32]:
from langchain.embeddings import OpenAIEmbeddings

In [35]:
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)   # model_name="ada"
text = "안녕하세요. 한도제한계좌의 한도를 해지하러 왔습니다."
text_embedding = embeddings.embed_query(text)

In [36]:
print(f"임베딩 길이 : {len(text_embedding)}")
print(f"샘플은 다음과 같습니다 : {text_embedding[:5]}...")

임베딩 길이 : 1536
샘플은 다음과 같습니다 : [0.004952843300998211, -0.040054306387901306, -0.008051319979131222, -0.026554523035883904, -0.02718837931752205]...


# Prompt

## Prompt Template

In [37]:
from langchain import PromptTemplate

In [38]:
template = """
금융용어에 관한 설명을 영어로 번역해 해주세요.
다음은 한도제한계좌에 관한 설명입니다.
- 한도제한계좌란 입출금이 자유로운 요구불계좌 신규 시 금융거래 목적에 대한 객관적인 증빙서류가 제출되지 않아 일반계좌 개설이 불가능한 경우에 개설되는 계좌이다.

번역된 결과는 원어민이 알아듣기 쉽도록 자연스러워야 합니다.
{keyword}란 무엇인가요?
"""

prompt = PromptTemplate(
    input_variables=["keyword"],
    template=template
)

## Prmopt Template Class

In [39]:
multiple_input_prompt = PromptTemplate(
    input_variables=['keyword', 'input_language', 'output_language'],
    template="Translate a {input_language}: {keyword}  to {output_language}: "
)

In [40]:
multiple_input_prompt.format(input_language="Korean", output_language="English", keyword="한도제한계좌")

'Translate a Korean: 한도제한계좌  to English: '

## Example Selector

In [42]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.llms import OpenAI

In [46]:
llm = OpenAI(model_name="text-davinci-003", openai_api_key=openai_api_key)
example_prompt = PromptTemplate(
    input_variables=['input', 'output'],
    template="Example Input: {input}\nExample Output: {output}"
)

In [47]:
# 명사가 발견되는 위치의 예
examples = [
  {"input": "pirate", "output": "ship"},
  {"input": "pilot", "output": "plane"},
  {"input": "driver", "output": "car"},
  {"input": "tree", "output": "ground"},
  {"input": "bird", "output": "nest"},
]

In [52]:
# SemanticSimilarityExampleSelector는 의미론적 의미에 따라 입력과 유사한 예제를 선택합니다.
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(openai_api_key=openai_api_key),    # 의미적 유사성을 측정하는 데 사용되는 임베딩을 생성하는 데 사용되는 임베딩 클래스
    FAISS,    # 임베딩을 저장하고 유사성 검색을 수행하는 데 사용되는 VectorStore 클래스
    k=2    # 생성할 예제 개수
)

In [54]:
similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,    # 예제 선택에 도움이 되는 개체
    example_prompt=example_prompt,
    prefix="Give the location an item is usually found in",    # 프롬프트 상단과 하단에 추가되는 사용자 지정 사항
    suffix="Input: {noun}\nOutput:",
    input_variables=['noun']
)

In [55]:
my_noun = "account"

In [56]:
print(similar_prompt.format(noun=my_noun))

Give the location an item is usually found in

Example Input: tree
Example Output: ground

Example Input: driver
Example Output: car

Input: account
Output:


In [57]:
llm(similar_prompt.format(noun=my_noun))

' bank'

## Output Parser

- Format instructions: 원하는 결과의 포맷을 지정하여 LLM에게 지시
- Parser: 원하는 텍스트를 출력 구조 (보통 json)을 추출하도록 함

In [58]:
# from langchain.output_parsers import CommaSeparatedListOutputParser

In [59]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI

In [60]:
llm = OpenAI(model_name="text-davinci-003", openai_api_key=openai_api_key)

In [61]:
# 응답을 어떻게 구성하고 싶은지 입력
response_schemas = [
    ResponseSchema(name="bad_string", description="This is a poorly formatted user input string"),
    ResponseSchema(name="good_string", description="This is your response, a reformatted response")
]

In [62]:
# 출력 구분 분석 방법
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [63]:
# 서식을 지정하려면 생성한 프롬프트 템플릿을 참조
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"bad_string": string  // This is a poorly formatted user input string
	"good_string": string  // This is your response, a reformatted response
}
```


In [64]:
template = """
You will be given a poorly formatted string from a user.
Reformat it and make sure all the words are spelled correctly
{format_instructions}
% USER INPUT:
{user_input}
YOUR RESPONSE:
"""

prompt = PromptTemplate(
  input_variables=["user_input"], partial_variables={"format_instructions": format_instructions},
  template=template
)

In [65]:
promptValue = prompt.format(user_input="welcom to califonya!")
print(promptValue)


You will be given a poorly formatted string from a user.
Reformat it and make sure all the words are spelled correctly
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"bad_string": string  // This is a poorly formatted user input string
	"good_string": string  // This is your response, a reformatted response
}
```
% USER INPUT:
welcom to califonya!
YOUR RESPONSE:



In [69]:
llm_output = llm(promptValue)
llm_output

'```json\n{\n\t"bad_string" : "welcom to califonya!",\n\t"good_string" : "Welcome to California!"\n}\n```'

In [70]:
output_parser.parse(llm_output)

{'bad_string': 'welcom to califonya!', 'good_string': 'Welcome to California!'}

# Index

Index는 LLM이 다른 소스에서 문서를 쉽게 가져올 수 있도록 하는 방법이다. 문서 작업을 위한 유틸리티 함수, 다양한 유형의 Index, 그리고 이러한 Index를 체이닝하여 사용한다.

In [71]:
# from langchain.document_loaders import HNLoader                 # 댓글 가져오기
# from langchain.document_loaders import UnstructuredURLLoader    # 사이트 로더
# from langchain.document_loaders import YoutubeLoader            # 유튜브 자막 불러오기

## Text Splitters

...

# Memory

기본적으로 체인과 에이전트는 상태를 저장하지 않는다. 즉, 쿼리가 수행되면 독립적으로 처리한다.  
Memory component는 LLM이 정보를 기억할 수 있도록 지원한다.

In [73]:
from langchain.memory import ChatMessageHistory
from langchain.chat_models import ChatOpenAI

In [74]:
chat = ChatOpenAI(openai_api_key=openai_api_key)

history = ChatMessageHistory()

history.add_ai_message("hi!")
history.add_user_message("what is the capital of france?")

In [75]:
history.messages

[AIMessage(content='hi!', additional_kwargs={}, example=False),
 HumanMessage(content='what is the capital of france?', additional_kwargs={}, example=False)]

# Chain
다양한 LLM을 호출하는데 사용되는 컴포넌트

## Simple Sequential Chains

In [108]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain

In [177]:
llm = OpenAI(temperature=1, openai_api_key=openai_api_key)

template = """
당신은 오타 검수자입니다. 다음의 문장을 교정해주세요.
% USER SENTENCE
{user_sentence}
YOUR RESPONSE:
"""

prompt_template = PromptTemplate(input_variables=["user_sentence"], template=template)

# 내 '문장' 체인을 보관
correction_chain = LLMChain(llm=llm, prompt=prompt_template)

template = """
당신은 금융전문 통역봇입니다. 문장이 주어졌을 때, 해당 문장을 영어로 쉽고 자연스럽게 번역해줘. 전문용어가 포함되어 있으면 그 용어에 대해서 간략하게 설명해줘.
% CORRECTED SENTENCE
{corrected_sentence}
TRANSLATED: 
"""

prompt_template = PromptTemplate(input_variables=["corrected_sentence"], template=template)

# 내 '번역' 체인 보관
translate_chain = LLMChain(llm=llm, prompt=prompt_template)

sent1 = "FACTA 등록에 필요한 납세자 번호를 알려주세엿."
sent2 = "이채한도가 제한되는 한도제한계좌입니당."
sent3 = "핸드폰이 본인 명의인가요?ㅎㅎ"
sent4 = "재직증면서 또는 회사에 근무한다는 증명서가 필요합니다."
sent5 = "농협은행 NH더하고나눔정기예금 가입 도와드릴까요?"

# overall_chain = SimpleSequentialChain(chains=[correction_chain, translate_chain], verbose=True)
# review = overall_chain.run(sent3)

In [178]:
# 중간과정 검수
for origin in [sent1, sent2, sent3, sent4, sent5]:
    corrected = correction_chain.run(origin)
    if len(corrected) != len(origin):
        corrected = origin
    translated = translate_chain.run(corrected)
    print(f"Origin : {origin}")
    print(f"Corrected : {corrected}")
    print(f"Translated: {translated}")
    print()

Origin : FACTA 등록에 필요한 납세자 번호를 알려주세엿.
Corrected : FACTA 등록에 필요한 납세자 번호를 알려주세요.
Translated: Please provide me the Taxpayer Identification Numbers (TINs) needed for FACTA registration.

Origin : 이채한도가 제한되는 한도제한계좌입니당.
Corrected : 이체한도가 제한되는 한도제한계좌입니다.
Translated: This is a limit-restriction account which limits your wire transfer limit.

Origin : 핸드폰이 본인 명의인가요?ㅎㅎ
Corrected : 핸드폰이 본인 명의인가요?ㅎㅎ
Translated: Is the phone in your own name?

Origin : 재직증면서 또는 회사에 근무한다는 증명서가 필요합니다.
Corrected : 재직증면서 또는 회사에 근무한다는 증명서가 필요합니다.
Translated: A proof of employment or working for a company is required.

Origin : 농협은행 NH더하고나눔정기예금 가입 도와드릴까요?
Corrected : 농협은행 NH더하고나눔정기예금 가입 도와드릴까요?
Translated: Can I help you open an NH-thePlus&Share regular deposit account at Nonghyup Bank? ThePlus&Share is a regular deposit account where customers save money regularly over a period of two month or more; the customers can then use the saved money as they wish.



## 요약 체인

In [129]:
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [130]:
loader = TextLoader("data/finance_words.txt")
documents = loader.load()

In [131]:
documents

[Document(page_content='한도계좌는 은행에서 신규로 입출금계좌를 만들때, 입금과 출금 한도를 제한하는 계좌이다. 정식 명칭은 금융거래한도제한계좌이지만, 이름이 길어서 보통, 금융거래한도계좌, 거래한도계좌, 한도계좌 등으로 줄여서 쓴다. 반대말은 일반계좌이다.', metadata={'source': 'data/finance_words.txt'})]

In [132]:
# splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=50)

In [133]:
# split
texts = text_splitter.split_documents(documents)

In [135]:
# map-reduce를 통해 긴 전체 문서를 요약한다.
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=True)
chain.run(texts)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"한도계좌는 은행에서 신규로 입출금계좌를 만들때, 입금과 출금 한도를 제한하는 계좌이다. 정식 명칭은 금융거래한도제한계좌이지만, 이름이 길어서 보통, 금융거래한도계좌, 거래한도계좌, 한도계좌 등으로 줄여서 쓴다. 반대말은 일반계좌이다."


CONCISE SUMMARY:

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


" 한도계좌는 금융거래한도제한계좌로, 은행에서 입출금계좌를 신규로 만들 때 입금과 출금 한도를 제한하는 계좌이다. 반대말은 일반계좌이다."


CONCISE SUMMARY:

> Finished chain.

> Finished chain.


' 한도계좌는 금융거래한도가 제한된 입출금계좌이고, 일반계좌는 한도를 갖지 않는 계좌이다.'

# Agents
사용자 입력에 따라 Agent는 여러 도구 중 호출할 수 있는 경우 도구를 결정할 수 있다.
Agent는 LLM을 사용하여 수행할 작업과 순서를 결정한다. 이 과정에서 도구를 사용하여 출력을 관찰하거나 사용자에게 반환할 수 있다.

## 구글 검색

In [ ]:
# # 구글 API 환경설정
# import os
# os.environ["GOOGLE_CSE_ID"] = ""
# os.environ["GOOGLE_API_KEY"] = ""


# from langchain.utilities import GoogleSearchAPIWrapper

# search = GoogleSearchAPIWrapper()
# search.run("Obama's first name?")